<a href="https://colab.research.google.com/github/cs145442/nlp-projects-with-tf2/blob/master/topic_classification_with_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers

     |████████████████████████████████| 675kB 4.7MB/s 
     |████████████████████████████████| 890kB 18.3MB/s 
     |████████████████████████████████| 1.1MB 35.5MB/s 
     |████████████████████████████████| 3.8MB 46.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ef1200caacbd9b9642374609cded70ca7566af87d136bcadb57384936ae2f417
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
# load the sentence-bert model from the HuggingFace model hub
from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F
tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')

In [0]:

def predict_labels(sentence: str, labels: list):
  # run inputs through model and mean-pool over the sequence
  # dimension to get sequence-level representations
  inputs = tokenizer.batch_encode_plus([sentence] + labels,
                                      return_tensors='pt',
                                      pad_to_max_length=True)
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']
  output = model(input_ids, attention_mask=attention_mask)[0]
  sentence_rep = output[:1].mean(dim=1)
  label_reps = output[1:].mean(dim=1)

  # now find the labels with the highest cosine similarities to
  # the sentence
  similarities = F.cosine_similarity(sentence_rep, label_reps)
  closest = similarities.argsort(descending=True)
  for ind in closest:
      print('label: ' + labels[ind] + '\t ' + 'similarity: ' + "{:.2e}".format(similarities[ind]))

In [0]:
labels = ['sports', 'late', 'sick', 'caring', 'anger']

In [0]:
sentences = ["Tom bought a ticket to witness his first ever football match in the indoor stadium. He went to the match and enjoyed a lot.",
             "Last night while returning from office, tom got drenched in the rain. The next morning, he had fever and had to take a day off from his work.",
             "Tom visits the old age home near Marathahalli every Saturday. He loves to spend time at the old age home by serving food, playing with them and making them laugh",
             "Tom was going to office in his brand-new car. At one of the red lights, a car banged his car from the back. Tom got out of his car and scolded the driver of the other car. ",
             "I am a social worker and have been taking multiple awareness initiatives during the current COVID-19 situation. I had an important meeting with the Chief Minister of Bengaluru at 10:00 am on 05 Jun 2020 to discuss on various possible options to increase the awareness level within the Government workers. I started from my house in my car at 8:30 am considering that in the current lock down situation the roads would be empty. However, I noticed that there was huge traffic at multiple points. By the time I reached the chief minister office, the meeting had already commenced with other stakeholders as per the scheduled timeline.",
             ]

In [22]:
for sentence in sentences:
  print("-"*50)
  print(sentence)
  predict_labels(sentence, labels)

--------------------------------------------------
Tom bought a ticket to witness his first ever football match in the indoor stadium. He went to the match and enjoyed a lot.
label: sports	 similarity: 1.06e-01
label: caring	 similarity: -1.88e-02
label: late	 similarity: -7.67e-02
label: anger	 similarity: -1.01e-01
label: sick	 similarity: -1.40e-01
--------------------------------------------------
Last night while returning from office, tom got drenched in the rain. The next morning, he had fever and had to take a day off from his work.
label: sick	 similarity: 1.96e-01
label: late	 similarity: 1.13e-01
label: anger	 similarity: 8.80e-02
label: sports	 similarity: -1.35e-01
label: caring	 similarity: -1.53e-01
--------------------------------------------------
Tom visits the old age home near Marathahalli every Saturday. He loves to spend time at the old age home by serving food, playing with them and making them laugh
label: caring	 similarity: 1.55e-01
label: anger	 similarity: 1